In [ ]:
import os 
import sys 

sys.path.insert(0, '..')

from process_bigraph import Composite
from process_bigraph.experiments.parameter_scan import RunProcess

from biosimulator_processes.processes.amici_process import AmiciUTC
from biosimulator_processes import CORE 
from biosimulator_processes.utils import plot_utc_outputs

In [ ]:
omex_dirpath = './examples/sbml-core/Elowitz-Nature-2000-Repressilator'
model_path = os.path.join(omex_dirpath, 'BIOMD0000000012_url.xml')
reports_path = os.path.join(omex_dirpath, 'reports.h5')

obs1 = ["floating_species_concentrations", "ci_protein"]
proc_addresses = ['copasi', 'tellurium', 'amici']
proc_address = 'amici'

amici_process_config = {
    'model': {'model_source': omex_dirpath},
    'time_config': {'step_size': 0.2, 'duration': 1000, 'num_steps': 5001}
}

In [ ]:
amici_process = AmiciUTC(config=amici_process_config)

In [ ]:
results = amici_process.update()

In [ ]:
results

In [ ]:
plot_utc_outputs(results, simulator_name='Amici')

In [ ]:


# Example usage:
file_path = os.path.join(omex_dirpath, 'simulation.sedml')
time_course_data = extract_uniform_time_course(file_path)
print(time_course_data)

In [ ]:
from zipfile import ZipFile

with ZipFile(omex_dirpath + '.omex', 'r') as f:
    f.extractall(path="/Users/alex/Desktop/uchc_work")

In [ ]:
!ls ~/desktop/uchc_work

In [ ]:
! cat ~/desktop/uchc_work/simulation.sedml